In [3]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import mysql.connector

# konektor ke database
db = mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='UnNgGrape'
        )
cursor = db.cursor()

# membaca data scrape sebagai patokan scraping
cursor.execute("SELECT * FROM scrape")
raw_scrape = cursor.fetchall()
for row in raw_scrape:
    print("----------------------------------------------------------------------------------------------------------------------------")
    print("Scrape for "+row[0])
    print("----------------------------------------------------------------------------------------------------------------------------")

    # memberi nama objek untuk data yg diambil sebagai untuk mempermudah
    main_link1 = row[1]
    main_link2 = row[2]
    tag_main = row[3]
    tag_main_part = row[4]
    tag_lowongan = row[5]
    tag_lowongan_part = row[6]
    tag_perusahaan = row[7]
    tag_perusahaan_part = row[8]
    tag_lokasi = row[9]
    tag_lokasi_part = row[10]
    raw_link = row[11]
    tag_deskripsi = row[12]
    tag_deskripsi_part = row[13]

    # scrape berdasarkan data keyword
    cursor.execute("SELECT * FROM keyword")
    raw_keyword = cursor.fetchall()
    for row2 in raw_keyword:
        # penyiapan scraping
        keyword = row2[0]
        main_link = main_link1 + keyword + main_link2
        print("_____________________________________________________________________________________________________")
        print("From keyword: "+keyword)
        print("_____________________________________________________________________________________________________")

        # menyiapkan scrape berdasarkan link yang sudah disiapkan
        r = Request(main_link, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'})
        response = urlopen(r).read()
        print(type(response))
        soup = BeautifulSoup(response, "lxml")
        print(type(soup))

        # memulai scrape
        jobList = soup.find_all(tag_main, tag_main_part)
        for p in jobList:
            lowongan = p.find(tag_lowongan, tag_lowongan_part).get_text()
            # menggunakan try except karena ada beberapa perusahaan yang dirahasiakan
            try:
                perusahaan = p.find(tag_perusahaan, tag_perusahaan_part).get_text()
            except:
                perusahaan = "Perusahaan Dirahasiakan"
            lokasi = p.find(tag_lokasi, tag_lokasi_part).get_text()
            link = raw_link + p.find('a').get('href')

            cursor.execute("SELECT * FROM lowongan WHERE link_lowongan = '" + link + "'")
            data = cursor.fetchall()
            # if else untuk pengecekan input data
            
            if data:
                print("Data exists")
            else:
                try:
                    #   scrape detail sesuai link
                    r2 = Request(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
                    response2 = urlopen(r2).read()
                    soup2 = BeautifulSoup(response2, "lxml")
                    
                    raw_deskripsi = soup2.find(tag_deskripsi, tag_deskripsi_part)
                    # increment deskripsi
                    deskripsi = ""

                    if raw_deskripsi is None:
                        print("raw_deskripsi Nonetype. link: "+ link)
                    else:
                        for string in raw_deskripsi.strings:
                            new_string  = deskripsi + " " + string
                            deskripsi = new_string
                except:
                    deskripsi = ""
        

                cursor.execute(
                "INSERT INTO lowongan(title_lowongan, nama_perusahaan, lokasi_perusahaan, deskripsi_lowongan, link_lowongan)"
                "VALUES ('"+ lowongan.replace("'", "").replace('"', '') +"', '"+ perusahaan.replace("'", "").replace('"', '') +"', '"+ lokasi.replace("'", "").replace('"', '') +"', '"+ deskripsi.replace("'", "").replace('"', '').replace("Informasi Penting Pastikan perusahaan yang kamu lamar resmi dengan memeriksa website dan lowongan kerja mereka. Read Less ", "") +"', '"+ link +"')"
                )
                print("Data has been added")

db.commit()
print("________________________________________DONE________________________________________")

----------------------------------------------------------------------------------------------------------------------------
Scrape for glints
----------------------------------------------------------------------------------------------------------------------------
_____________________________________________________________________________________________________
From keyword: teknik
_____________________________________________________________________________________________________
<class 'bytes'>
<class 'bs4.BeautifulSoup'>
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
_____________________________________________________________________________________________________
F